В данном ноутбуке большая часть кода скопирована с целью переноса данных (Задание 1 и Задание 2 помечено внутри кода, выделено крупным шрифтом)
С заданием 3 я пока не успела разобраться, к сожалению 

Постройте график зависимости весов всех признаков от lambda в L2-регуляризации на основе данных из урока.

In [7]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]])

In [9]:
y = [45, 55, 50, 55, 60, 35, 75, 80, 50, 60]

Стандартизация

In [19]:
X_st = X.copy().astype(np.float64)
X1_mean = X[1].mean()
X1_std = X[1].std()
print(X1_mean, X1_std)

3.0 2.8284271247461903


In [20]:
X_st[1] = (X[1] - X1_mean) / X1_std
X_st

array([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00],
       [-7.07106781e-01, -7.07106781e-01, -3.53553391e-01,
         7.07106781e-01,  0.00000000e+00, -1.06066017e+00,
         7.07106781e-01,  2.47487373e+00, -7.07106781e-01,
        -3.53553391e-01],
       [ 5.00000000e+02,  7.00000000e+02,  7.50000000e+02,
         6.00000000e+02,  1.45000000e+03,  8.00000000e+02,
         1.50000000e+03,  2.00000000e+03,  4.50000000e+02,
         1.00000000e+03],
       [ 1.00000000e+00,  1.00000000e+00,  2.00000000e+00,
         1.00000000e+00,  2.00000000e+00,  1.00000000e+00,
         3.00000000e+00,  3.00000000e+00,  1.00000000e+00,
         2.00000000e+00]])

In [21]:
def calc_std_fit(x):
    res = (x - x.mean()) / x.std()
    return res

In [22]:
X_st[2] = calc_std_fit(X[2])

In [23]:
X_st[3] = calc_std_fit(X[3])
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [-0.70710678, -0.70710678, -0.35355339,  0.70710678,  0.        ,
        -1.06066017,  0.70710678,  2.47487373, -0.70710678, -0.35355339],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [-0.89625816, -0.89625816,  0.38411064, -0.89625816,  0.38411064,
        -0.89625816,  1.66447944,  1.66447944, -0.89625816,  0.38411064]])

SGD (Stochastic gradient descent)

In [15]:
def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

In [26]:
def eval_sgd_model(X, y, iterations, qty_in_batch, alpha=1e-4):
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    n_batch = n // qty_in_batch
    
    if n % qty_in_batch != 0:
        n_batch += 1
        
    for i in range(1, iterations+1):
        for b in range(n_batch):
            start_ = qty_in_batch*b
            end_ = qty_in_batch*(b+1)
#             print(b, n_batch, start_, end_)
            X_tmp = X[:, start_ : end_]
            y_tmp = y[start_ : end_]

            y_pred_tmp = np.dot(W, X_tmp)
            err = calc_mse(y_tmp, y_pred_tmp)

            W -= (alpha * (1/n * 2 * np.dot((y_pred_tmp - y_tmp), X_tmp.T)))
        
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [27]:
eval_sgd_model(X_st, y, iterations=5000, qty_in_batch=4, alpha=1e-2)

500 [56.50790436  6.00130349  0.99825395  5.48682878] 13.112836710261227
1000 [56.51048053  6.21037999  0.12261155  6.18358248] 11.93636224592878
1500 [56.51056972  6.27640228 -0.18948231  6.44004154] 11.520561637162444
2000 [56.51060212  6.29968222 -0.30160564  6.53259698] 11.372180248622625
2500 [56.51061379  6.30803279 -0.34193305  6.56590793] 11.31900626376039
3000 [56.51061799  6.31103558 -0.35643997  6.57789193] 11.299906473611742
3500 [56.5106195   6.31211573 -0.36165864  6.58220307] 11.293039418582838
4000 [56.51062004  6.3125043  -0.36353599  6.58375395] 11.290569584181288
4500 [56.51062024  6.31264408 -0.36421135  6.58431186] 11.28968115661532
5000 [56.51062031  6.31269436 -0.3644543   6.58451257] 11.28936156393927


array([56.51062031,  6.31269436, -0.3644543 ,  6.58451257])

# L1 регуляризация (Задание 3)

f(x, W) + (lambda_/2) * np.sum(np.abs(W))

In [51]:
def eval_model_reg1(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        y_pred = np.dot(W, X)
        err = calc_mse(y, y_pred)
#В отличие от L2 регуляризации, мы меняем производную квадрата на производную модуля 
        W -= alpha * (1/n * 2 * np.dot((y_pred - y), X.T) + lambda_ * W/np.abs(W)) 
#         if i % (iterations / 10) == 0:
#             print(i, W, err)
    return W

In [52]:
W = eval_model_reg1(X_st, y, iterations=5000, alpha=1e-2, lambda_=1e-4)

In [77]:
eval_model(X_st, y, iterations=5000, alpha=1e-2)

500 [56.49767254  5.83477719  1.36781865  5.25424451] 25.413829625514797
1000 [56.4999999   6.17124081  0.25269258  6.07416416] 24.969784391475397
1500 [56.5         6.2575128  -0.13774522  6.39128671] 24.913059475565497
2000 [56.5         6.28678323 -0.27772011  6.50655795] 24.905712417850083
2500 [56.5         6.29722767 -0.32807425  6.54810646] 24.904760553791704
3000 [56.5         6.30098239 -0.34619733  6.56306445] 24.904637232271696
3500 [56.5         6.30233363 -0.3527205   6.5684486 ] 24.904621254991206
4000 [56.5         6.30281999 -0.35506846  6.57038658] 24.904619185007856
4500 [56.5         6.30299505 -0.35591359  6.57108415] 24.904618916825108
5000 [56.5         6.30305806 -0.35621778  6.57133523] 24.90461888207993


array([56.5       ,  6.30305806, -0.35621778,  6.57133523])

Задание 2

Мы можем одновременно использовать, как нормализацию, так и стандартизацию, однако в этом нет особого смысла. Работать будет тот метод, который идет вторым (ну или первый, если мы назовем ряд переменных по-разному и будем использовать какой-то один в дальнейшем анализе) 